<a href="https://colab.research.google.com/github/Osvaldo1417/Administracion-de-Riesgos-Financieros/blob/master/Tarea_1_3_Riesgo_de_Mercado_Simulaci%C3%B3n_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1.3 - Riesgo de Mercado
## Simulación MonteCarlo: Cholesky y Componentes Principales
- Cedeño Ramírez Osvaldo
- Felipe Santiago Edson Yair
- Morelos Granados José Carlos
- Pacheco Tovar Alejandra Citlalli


In [1]:
%load_ext rpy2.ipython
!rm -rf Administracion-de-Riesgos-Financieros
!git clone "https://github.com/Osvaldo1417/Administracion-de-Riesgos-Financieros/"

Cloning into 'Administracion-de-Riesgos-Financieros'...
remote: Enumerating objects: 851, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 851 (delta 10), reused 0 (delta 0), pack-reused 827
Receiving objects: 100% (851/851), 41.48 MiB | 18.81 MiB/s, done.
Resolving deltas: 100% (542/542), done.


In [ ]:
%%R
install.packages("quantmod")
library(quantmod)

install.packages("data.table")
library(data.table)

install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)

library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

## Funciones

In [3]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

genera_esc <- function(lamda,w0,n){
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  return(p_esc)
}

# Carga de Datos

In [111]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

## Acciones y Divisas

In [112]:
%%R
acc_names <- c("America","Carso","Walmart")
div_names <- c("Euros","Libras","Dolares")

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Sym_acciones<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_acc=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
Sym_divisas<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_div=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

In [113]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_div),t(pos_acc))
start_date=fval-nh #fecha inicial
#Creación del objeto para guardar los datos
acciones<-new.env()
divisas<-new.env()

#obtener los datos
getSymbols.yahoo(Sym_acciones,env=acciones,from=start_date,to=fval)
getSymbols.yahoo(Sym_divisas,env=divisas,from=start_date,to=fval)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(acciones,align='remove.na',fill.gaps = T)
bt.prep(divisas,align='remove.na',fill.gaps=T)

precios_acc = acciones$prices
precios_div = divisas$prices

precios_div=cbind(precios_div[,1]*precios_div[,3],
                     precios_div[,2]*precios_div[,3],precios_div[,3])

precios_acc_div = merge(precios_acc,precios_div,join = "inner")
tail(precios_acc_div)

           AMXL.MX GCARSOA1.MX WALMEX.MX EURUSD.X GBPUSD.X USDMXN.X
2022-03-23   20.73       73.71     77.99 22.35971 26.89405 20.27220
2022-03-24   20.91       75.61     78.56 22.26207 26.71136 20.22664
2022-03-25   21.00       75.77     78.59 22.09215 26.47899 20.06630
2022-03-28   20.72       75.81     79.83 21.99689 26.39227 20.03015
2022-03-29   20.70       74.29     78.58 22.08800 26.32673 20.09125
2022-03-30   20.75       73.53     80.34 22.14488 26.14256 19.96560


In [114]:
%%R
aux2=data.table(Date=as.Date(index(precios_acc_div)),coredata(precios_acc_div))
colnames(aux2) <- c("Date","America","Carso","Walmart","Euros","Libras","Dolares")
tail(aux2)

         Date America Carso Walmart    Euros   Libras  Dolares
1: 2022-03-23   20.73 73.71   77.99 22.35971 26.89405 20.27220
2: 2022-03-24   20.91 75.61   78.56 22.26207 26.71136 20.22664
3: 2022-03-25   21.00 75.77   78.59 22.09215 26.47899 20.06630
4: 2022-03-28   20.72 75.81   79.83 21.99689 26.39227 20.03015
5: 2022-03-29   20.70 74.29   78.58 22.08800 26.32673 20.09125
6: 2022-03-30   20.75 73.53   80.34 22.14488 26.14256 19.96560


## Bonos

In [115]:
%%R
#CETES
tasa_guber_bcc="Administracion-de-Riesgos-Financieros/tasa_guber.txt"
plazos_bcc=cbind(180)
contratos_bcc=cbind(1)
nominal_bcc=1500 #CETE

# BONO M
tasa_fija_bm="Administracion-de-Riesgos-Financieros/tasa_yield.txt"
tfcupon_bm=cbind( 0.065) #Tasafija del cupón
plazos_bm=cbind( 3600) #Vencimiento del bono
plazocupon_bm=cbind(360) #plazos fijos de cada cupón
contratos_bm=cbind(1) #posición invertida
nominal_bm=1000

#BONDES D
base_tasa_desc="Administracion-de-Riesgos-Financieros/tasa_guber_st.txt"
base_tasa_fondeo="Administracion-de-Riesgos-Financieros/tfondeo.txt"
plazo_bdm= 707 #Vencimiento del bono
plazocupon_bdm = 28 #plazos_bdm fijos de cada cupón
contratos_bdm = -1 #posición invertida
nominal_bdm = 1000

In [116]:
%%R
#Bono Cupon CERO
data<-read.table(tasa_guber_bcc)
n<-nrow(data)
m_orig=ncol(data)
X_orig_bcc=mutate(data.frame(data[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bcc=X_orig_bcc%>%select(-V1)
nodos_bcc=data.frame(data[1,2:m_orig])

## BONO M
data1<-read.table(tasa_fija_bm)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig_bm=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bm=X_orig_bm%>%select(-V1)
nodos_bm=data.frame(data1[1,2:m_orig])
n=n-1

##BONDE
data1<-read.table(tasa_guber_bcc)
n_bd<-nrow(data1)
m_bd=ncol(data1)
X1_orig=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X1_orig=X1_orig%>%select(-V1)
nodos1_bd=data.frame(data1[1,2:m_bd])
n_bd=n_bd-1

data2<-read.table(base_tasa_fondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,])
X2_orig_bd=mutate(X2_orig_bd, Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(V2))
X2_orig_bd=X2_orig_bd%>%select(-V1)
tfh=seq(min(X2_orig_bd$Date), max(X2_orig_bd$Date), "days")
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

data3<-read.table(base_tasa_desc)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := Date][order(-Date)]
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf]

tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

## Futuros

In [117]:
%%R
#FUTUROS DE TDC
bext="Administracion-de-Riesgos-Financieros/tasa_libor.txt" 
bdom="Administracion-de-Riesgos-Financieros/tasa_fwd.txt"
SymbolsFX_ftdc_find <- c("^MXX","USDMXN=X")
plazos_fowd = cbind(5)
contratos_fowd = cbind(100)
kst_fwd=cbind(20.83)
nominal_fwd=cbind(1)
trlib=1

#FUTUROS DE IPC
btsp="Administracion-de-Riesgos-Financieros/tasa_guber.txt"
plazos_fwd_ind= cbind(53) 
contratos_fwd_ind= cbind(-50)
kst_fwd_ind=cbind(49525)
nominal_fwd_ind=cbind(1)

In [118]:
%%R
### CARGA DE DATOS TDC
data1<-read.table(bext)
data2<-read.table(bdom)

n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1

###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])

####MATRICES DEL MISMO TAMAÑO
x1_ftdc=as.data.table(mutate(data1[2:n1,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n1,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)


# CARGA DE DATOS DE  FORWARD DE IPC
data1<-read.table(btsp)
n<-nrow(data1)
m_gov=ncol(data1)
x_orig_gov=as.data.table(mutate(data1[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data1[1,2:m_gov])

#Cargar los símbolos de yahoo finance para EQ
dataEnvFX_EQ<-new.env()
getSymbols.yahoo(SymbolsFX_ftdc_find,env=dataEnvFX_EQ,from=start_date, to=fval)
bt.prep(dataEnvFX_EQ,align='remove.na',fill.gaps=T)

X3_find=data.table(Date=as.Date(index(dataEnvFX_EQ$prices[,1])),coredata(dataEnvFX_EQ$prices[,1]))
X3_find = X3_find[order(-Date)]

###Para Dolar
X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_EQ$prices[,2])),coredata(dataEnvFX_EQ$prices[,2]))
X3_ftdc = X3_ftdc[order(-Date)]

## Swap's

In [119]:
%%R
btasadesc_sw="Administracion-de-Riesgos-Financieros/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="Administracion-de-Riesgos-Financieros/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.066,0.059) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(16000000,-12000000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

In [120]:
%%R
#CARGA DE DATOS PARA SWAP
data1<-read.table(btasadesc_sw)###tasa tie
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])

data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])

## Opciones

In [121]:
%%R
#OPCIONES
base_desc_opc="Administracion-de-Riesgos-Financieros/tasa_TIIE_SW_OP.txt"
base_spot_opc="Administracion-de-Riesgos-Financieros/tasa_DIRS_SW_OP.txt"
base_vot_opc="Administracion-de-Riesgos-Financieros/tvoltiie_opc.txt"
plazos_opc=cbind( 1700, 700) #T-t
pr_opc=28 #plazo de referencia  
dct_opc=360 #d_base
cp_opc=cbind(0,1) # 0 si es call (cap) o put (floor)
K_opc=cbind( 0.058, 0.06)
contratos_opc=cbind(1000, 500)
nominal_opc=1
cs_opc=1 #1 si es continua la tasa 0 si es simple

In [122]:
%%R
#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
#carga de rho	
data1<-read.table(base_desc_opc)
n<-nrow(data1)
m1_orig_opc=ncol(data1)
x1_orig_opc=data.table(mutate(data1[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x1_orig_opc=x1_orig_opc%>%select(-V1)
nodos1_opc=data.frame(data1[1,2:m1_orig_opc])

#carga de tasas spot
data2<-read.table(base_spot_opc)
n<-nrow(data2)
m2_orig_opc=ncol(data2)
x2_orig_opc=data.table(mutate(data2[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x2_orig_opc=x2_orig_opc%>%select(-V1)
nodos2_opc=data.frame(data2[1,2:m2_orig_opc])

#carga de volatilidades de spot
data3<-read.table(base_vot_opc)
n<-nrow(data3)
m3_orig_opc=ncol(data3)
x3_orig_opc=data.table(mutate(data3[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x3_orig_opc=x3_orig_opc%>%select(-V1)
nodos3_opc=data.frame(data3[1,2:m3_orig_opc])

# Integración de insumos

In [123]:
%%R
fechas=data.table(Date=as.Date(aux2[X_orig_bcc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc
fechas=data.table(Date=as.Date(fechas[X_orig_bm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm
fechas=data.table(Date=as.Date(fechas[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd

fechas=data.table(Date=as.Date(fechas[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find
fechas=data.table(Date=as.Date(fechas[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext
fechas=data.table(Date=as.Date(fechas[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom
fechas=data.table(Date=as.Date(fechas[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc

fechas=data.table(Date=as.Date(fechas[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc
fechas=data.table(Date=as.Date(fechas[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup

fechas=data.table(Date=as.Date(fechas[x1_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc
fechas=data.table(Date=as.Date(fechas[x2_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc, opc spot
fechas=data.table(Date=as.Date(fechas[x3_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc, opc spot, opc volt

print(head(fechas))
print(tail(fechas))

         Date
1: 2022-03-30
2: 2022-03-29
3: 2022-03-28
4: 2022-03-25
5: 2022-03-24
6: 2022-03-23
         Date
1: 2021-04-12
2: 2021-04-09
3: 2021-04-08
4: 2021-04-07
5: 2021-04-06
6: 2021-04-05


## Segreración de Datos

In [124]:
%%R
n=nrow(fechas) #Historia de todos
n

[1] 253


### Acciones y Divisas

In [125]:
%%R
#historia de acciones y divisas
precios_acc_div = fechas[aux2,on=.(Date),nomatch=0][order(-Date)]
precios_acc_div = precios_acc_div%>%select(-Date)
precios_acc_div

     America Carso Walmart    Euros   Libras  Dolares
  1:   20.75 73.53   80.34 22.14488 26.14256 19.96560
  2:   20.70 74.29   78.58 22.08800 26.32673 20.09125
  3:   20.72 75.81   79.83 21.99689 26.39227 20.03015
  4:   21.00 75.77   78.59 22.09215 26.47899 20.06630
  5:   20.91 75.61   78.56 22.26207 26.71136 20.22664
 ---                                                 
249:   14.11 57.39   63.94 23.94925 27.60465 20.09894
250:   14.41 59.39   66.11 23.99821 27.77805 20.21130
251:   14.10 59.62   63.97 23.95562 27.90382 20.17446
252:   14.17 58.93   65.11 23.99920 28.25341 20.30573
253:   14.28 57.78   66.86 23.89553 28.08468 20.30522


### Bonos

In [126]:
%%R
#historia de curva gubernamental
X_orig_bcc=fechas[X_orig_bcc,on=.(Date),nomatch=0][order(-Date)]
X_orig_bcc=X_orig_bcc%>%select(-Date)

## Historia Bono M
X_orig_bm=fechas[X_orig_bm,on=.(Date),nomatch=0][order(-Date)]
X_orig_bm=X_orig_bm%>%select(-Date)

#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=fechas[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)

X2_orig_bd=fechas[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)

X3_orig_bd=fechas[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

### Futuros

In [127]:
%%R
#historia de curva gubernamental
x_orig_gov=fechas[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curvas de forward tdc
x1_ftdc=fechas[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=fechas[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=fechas[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
X3_find=fechas[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

### Swap's

In [128]:
%%R
X1_orig_sw=fechas[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=fechas[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

### Opciones

In [129]:
%%R
#historia de opciones
#Taas vp
x1_orig_opc=fechas[x1_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x1_orig_opc=x1_orig_opc%>%select(-Date)/100

#Tasa spot
x2_orig_opc=fechas[x2_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x2_orig_opc=x2_orig_opc%>%select(-Date)/100

#Volt
x3_orig_opc=fechas[x3_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x3_orig_opc=x3_orig_opc%>%select(-Date)

# Valuación de Instrumentos

## Acciones y Divisas

In [130]:
%%R
x0_acc_div <- precios_acc_div[1,]
V0_acc_div <- cbind(t(pos_acc),t(pos_div))*x0_acc_div
m_acc <- length(pos_acc)
m_div <- length(pos_div)

## Bonos

### CETES

In [131]:
%%R
m=ncol(plazos_bcc)
x_bcc=matrix(0,n,m)
for (i in 1:(n))
{
  x_bcc[i,]=if(itpl==0){approx(as.matrix(nodos_gov),as.matrix(x_orig_gov[i,]),plazos_bcc,rule=2)$y}else{talamb(as.matrix(nodos_gov),as.matrix(x_orig_gov[i,]),plazos_bcc)}
}

x0_bcc=x_bcc[1,]

#función
bonocupcero = function(i,t){
  1/(1+i*t/360)
}

V0_bcc=as.matrix(bonocupcero(x0_bcc,plazos_bcc))*contratos_bcc*nominal_bcc #Valor actual de cada bono
VT0_bcc=sum(V0_bcc) #Valor total del portafolio al tiempo 0

### Bono M

In [132]:
%%R
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:n){
  X_bm[i,]=if(itpl==0){approx(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm,rule=2)$y}else{talamb(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm)}
}

#Función de valoración por tasa yield
bonoMyield=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm){
  N=as.integer(plazos_bm/plazocupon_bm)+1   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  p1=plazos_bm-plazocupon_bm*(N-1)   
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

x0_bm=t(as.matrix(X_bm[1,])) #tasas de descuento valor actual
V0_bm=bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)

### BONDE

In [133]:
%%R
#X2_pr=fechas[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr=X2_orig_bd
m=1

N_bd=as.integer(plazo_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazo_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m)){
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n)){
  Xvp_bd[i,]=if(itpl==0){approx(as.matrix(nodos_gov),as.matrix(x_orig_gov[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos_gov),as.matrix(x_orig_gov[i,]),VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm)}  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 

V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0_bd

          [,1]
[1,] -942.9513


## Futuros

### Tipo de Cambio

In [134]:
%%R
m=ncol(plazos_fowd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)


for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(as.matrix(nodos1_ftdc),as.matrix(x1_ftdc[j,]),plazos_fowd,rule=2)$y}else{talamb(as.matrix(nodos1_ftdc),as.matrix(x1_ftdc[j,]),plazos_fowd)}
  X2_fwtdc[j,]=if(itpl==0){approx(as.matrix(nodos2_ftdc),as.matrix(x2_ftdc[j,]),plazos_fowd,rule=2)$y}else{talamb(as.matrix(nodos2_ftdc),as.matrix(x2_ftdc[j,]),plazos_fowd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fowd/180)-1)*360/plazos_fowd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)
X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)

V0_fwtdc=futuroTC(plazos_fowd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fowd*nominal_fwd

### Indices

In [135]:
%%R
m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)

for (j in 1:n)
{
  X2_fwind[j,]=if(itpl==0){approx(as.matrix(nodos_gov),as.matrix(x_orig_gov[j,]),plazos_fwd_ind,rule=2)$y}else{talamb(as.matrix(nodos_gov),as.matrix(x_orig_gov[j,]),plazos_fwd_ind)}
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X2_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind

## Swap's

In [136]:
%%R
nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw

plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}

Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(as.matrix(nodosvp),as.matrix(curvavp[i,]),VTplazos_sw,rule=2)$y}else{talamb(as.matrix(nodosvp),as.matrix(curvavp[i,]),VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_sw,rule=2)$y}else{talamb(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_swc, rule=2)$y}else{talamb(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}


V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)

## Opciones

In [137]:
%%R
m=ncol(plazos_opc)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(as.matrix(nodos1_opc),as.matrix(x1_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos1_opc),as.matrix(x1_orig_opc[i,]),plazos_opc)}
  x2tc[i,]=if(itpl==0){approx(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),plazos_opc)}
  x2tl[i,]=if(itpl==0){approx(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),(plazos_opc+pr_opc),rule=2)$y}else{talamb(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),(plazos_opc+pr_opc))}
  x3[i,]=if(itpl==0){approx(as.matrix(nodos3_opc),as.matrix(x3_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos3_opc),as.matrix(x3_orig_opc[i,]),plazos_opc)}
  
  x2[i,]=((1+x2tl[i,]*(plazos_opc+pr_opc)/360)/(1+x2tc[i,]*(plazos_opc)/360)-1)*360/pr_opc
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_opc=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_opc=opctint(x01,x02,K_opc,x03,plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio

# Simulación MonteCarlo

## Integración de Factores

In [138]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(precios_acc_div) #acciones y divisas
n_if[2]=ncol(x_bcc) #cetes
n_if[3]=ncol(X_bm) #bonos m
n_if[4]=ncol(X_bd_ext) #bonde
n_if[5]=ncol(X_futtdc) #fut tdc
n_if[6]=ncol(X_futind) #fut ind
n_if[7]=ncol(X_sw) #swaps
n_if[8]=ncol(X_opc) #opciones tasa de interés

#valor del portafolios

V0_port=cbind(V0_acc_div,V0_bcc, V0_bm, V0_bd, V0_fwtdc, V0_fwind, V0_sw, V0_opc) #contrato
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(precios_acc_div,x_bcc,X_bm,X_bd_ext,X_futtdc,X_futind,X_sw,X_opc) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

In [139]:
%%R
MonteCarlo <- function(ChCP,normempi,Ns,DeltaX_port,eta){
  mT=ncol(DeltaX_port)
  VarDeltaX=if(ChCP==0){cor(DeltaX_port)}else{var(DeltaX_port)*(ncol(DeltaX_port)/(ncol(DeltaX_port)-1))}

  if(ChCP==0) {#Cholesky
    CVarDeltaX=as.matrix(chol(VarDeltaX,pivot=TRUE))
    CVarDeltaX[is.nan(CVarDeltaX)] <- 0 #quitamos NaN
    CVarDeltaX[is.na(CVarDeltaX)] <- 0 #quitamos Na
    M=matrix(0, Ns,mT)
    if(normempi==1){
      for (i in 1:mT){
        M[,i]=quantile(DeltaX_port[,i],runif(Ns))
      }
      Met='Cholesky Empírico'
    } else	{
      for (i in 1:mT){
        M[,i]=rnorm(Ns)*sd(DeltaX_port[,i])*(mT/(mT-1))+mean(DeltaX_port[,i])
      }
      Met='Cholesky Normal'
    }
    DeltaX_s=M%*%CVarDeltaX  #Factores de riesgo simulados
  } else { #componentes principales
    G<-eigen(VarDeltaX)      #eigenvectores
    g1<-G$values                  #eigenvalores
    g1p=g1/sum(g1)
  
    g1pac=cumsum(g1p) #suma acumulada de varianza
  
    k=which.max(g1pac>eta) #valor mínimo k
  
    y=as.matrix((DeltaX_port-colMeans(DeltaX_port)))%*%G$vectors[,1:k] #componentes principales
    M=matrix(0, Ns,k)
    if(normempi==0){
      for (i in 1:k){
        M[,i]=rnorm(Ns)*sd(y[,i])+mean(y[,i])
      }
      Met='Componentes Principales Normales'
    } else	{
      for (i in 1:k){
        M[,i]=quantile(y[,i],runif(Ns))
      }
      Met='Componentes Principales Empíricos'
    }
    DeltaX_s=M%*% t(G$vectors[,1:k]) #Factores de riesgo simulados
  }
  ret <- list()
  ret[[1]] <- Met
  ret[[2]] <- DeltaX_s
  return(ret)
}

In [255]:
%%R
#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(X_port[1:(n-1),]/X_port[2:(n),]-1)
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=10000 #Definimos número de simulaciones
alpha=0.98 #Nivel de Confianza para las medidas de riesgo
ChCP=1  #0 Cholesky, 1 Componentes Principales
normempi=1 #0 normal, 1 empírico
eta=.85 #Varianza explicada  (sólo aplica para Componentes Principales)

aux <- MonteCarlo(ChCP,normempi,Ns,DeltaX_port,eta)
Met <- aux[[1]]
DeltaX_s <- aux[[2]]
print(Met)

#Ms1='Se explica el' 
#Ms2= 'de varianza'
#Ms3= 'de suma de var-cov'

#print(Ms1)
#sum(diag(var(DeltaX_s)))/sum(diag(var(DeltaX_port))) #Comprobación de matriz de varianza-cov vs original
#print(Ms2)

#print(Ms1)
#(matrix(1,1,ncol(DeltaX_s))%*%var(DeltaX_s)%*%t(matrix(1,1,ncol(DeltaX_s)))/matrix(1,1,ncol(DeltaX_s))%*%var(DeltaX_port)%*%t(matrix(1,1,ncol(DeltaX_s))))[1] #Comprobación de matriz de varianza-cov vs original
#print(Ms3)
#print(mT)
#print(k) #número de variables simuladas CP

[1] "Componentes Principales Empíricos"


# Medición de Riesgo



In [256]:
%%R
Met

[1] "Componentes Principales Empíricos"


## Acciones y Divisas

In [257]:
%%R
m=n_if[1]   #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m) #valor simulado a nivel contrato de acciones y tdc
Vfr1_acc_div=matrix(0,Ns,m_div) #valor simulado a nivel contrato de tdc
Vfr2_acc_div=matrix(0,Ns,m_acc) #valor simulado a nivel contrato de acciones

PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_div) 
PGfr2_acc_div=matrix(0,Ns,m_acc)

PGT_acc_div=matrix(0,Ns,1) #TOTAL ACC Y DIV
PGfr1T_acc_div=matrix(0,Ns,1) #total factor riesgo divisas
PGfr2T_acc_div=matrix(0,Ns,1) #total factor riesgo acciones

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=precios_acc_div[1,]   #PASO CLAVE

for (i in 1:Ns){
  X_s_acc_div[i,]=as.matrix(x0_acc_div*(1+DeltaX_s_acc_div[i,]))
  X_s_acc_div[i,] = cbind(X_s_acc_div[i,(m_acc+1):(m_acc+m_div)],X_s_acc_div[i,1:m_acc])
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_div),t(pos_acc))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_div)*X_s_acc_div[i,1:m_div]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_acc)*X_s_acc_div[i,(m_div+1):(m_div+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-cbind(V0_acc_div[,(m_acc+1):(m_acc+m_div)],V0_acc_div[,1:m_acc]))
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,(m_acc+1):(m_div+m_acc)])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,1:m_acc])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_div)
VaRfr2_acc_div=matrix(0,1,m_acc)

CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_div)
CVaRfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m)){
  VaRCont_acc_div[i]=quantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  if (i<=m_div){
    VaRfr1_acc_div[i]=quantile(PGfr1_acc_div[,i],1-alpha,Ns)
    CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
  if (i<=m_acc){  
    VaRfr2_acc_div[i]=quantile(PGfr2_acc_div[,i],1-alpha,Ns)
    CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
}


#VaR Total
VaRTotal_acc_div=quantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
VaRTotalfr1_acc_div=quantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
VaRTotalfr2_acc_div=quantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

In [258]:
%%R
VaR_acc_div <- rbind(VaRCont_acc_div,CVaRCont_acc_div)
rownames(VaR_acc_div) <- c("VaR","CVaR")
colnames(VaR_acc_div) <- c(div_names,acc_names)

print("VaR de acciones y divisas")
print(VaR_acc_div)

VaRT_acc_div <- rbind(cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div),cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div))
rownames(VaRT_acc_div) <- c("VaR","CVaR")
colnames(VaRT_acc_div) <- c("Total","Divisas","Acciones")

print("VaR Total de acciones y divisas")
print(VaRT_acc_div)

[1] "VaR de acciones y divisas"
         Euros    Libras   Dolares   America     Carso   Walmart
VaR  -20.60366 -21.64630 -65.59278 -1142.584 -2729.956 -813.5986
CVaR -27.05864 -27.31442 -78.97219 -1360.317 -3233.788 -958.8093
[1] "VaR Total de acciones y divisas"
         Total   Divisas  Acciones
VaR  -2619.184 -68.52375 -2620.145
CVaR -3105.404 -82.41651 -3122.039


## Bonos

In [259]:
%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)#Fondeo
Vfr2_bd=matrix(0,Ns,m)#Guber
Vfr3_bd=matrix(0,Ns,m)#Desc
PG_bd=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_bd=matrix(0,Ns,m)
PGfr2_bd=matrix(0,Ns,m)
PGfr3_bd=matrix(0,Ns,m)
PGT_bd=matrix(0,Ns,1)
PGfr1T_bd=matrix(0,Ns,1)
PGfr2T_bd=matrix(0,Ns,1)
PGfr3T_bd=matrix(0,Ns,1)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*(1+DeltaX_s_bd[i,])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  PG_bd[i,]=V_bd[i,]-V0_bd
  PGfr1_bd[i,]=Vfr1_bd[i,]-V0_bd
  PGfr2_bd[i,]=Vfr2_bd[i,]-V0_bd
  PGfr3_bd[i,]=Vfr3_bd[i,]-V0_bd
  PGT_bd[i,]=sum(PG_bd[i,])
  PGfr1T_bd[i,]=sum(PGfr1_bd[i,])
  PGfr2T_bd[i,]=sum(PGfr2_bd[i,])
  PGfr3T_bd[i,]=sum(PGfr3_bd[i,])
}

#VaR por posición
VaRCont_bd=matrix(0,1,m)
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
CVaRCont_bd=matrix(0,1,m)
CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
for (i in (1:m)){
  VaRCont_bd[i]=quantile(PG_bd[,i],1-alpha,Ns)
  VaRfr1_bd[i]=quantile(PGfr1_bd[,i],1-alpha,Ns)
  VaRfr2_bd[i]=quantile(PGfr2_bd[,i],1-alpha,Ns)
  VaRfr3_bd[i]=quantile(PGfr3_bd[,i],1-alpha,Ns)
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bd[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bd[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bd[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_bd[i]= mean(merge(which(PG_bd[,i]<VaRCont_bd[i]),cbind(seq(1,Ns),PG_bd[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total
VaRTotal_bd=quantile(PGT_bd,1-alpha,Ns)
CVaRTotal_bd= mean(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])
VaRTotalfr1_bd=quantile(PGfr1T_bd,1-alpha,Ns)
CVaRTotalfr1_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])
VaRTotalfr2_bd=quantile(PGfr2T_bd,1-alpha,Ns)
CVaRTotalfr2_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])
VaRTotalfr3_bd=quantile(PGfr3T_bd,1-alpha,Ns)
CVaRTotalfr3_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])

In [260]:
%%R
VaR_bd <- rbind(cbind(VaRTotal_bd,VaRTotalfr1_bd,VaRTotalfr2_bd,VaRTotalfr3_bd),cbind(CVaRTotal_bd,CVaRTotalfr1_bd,CVaRTotalfr2_bd,CVaRTotalfr3_bd))
colnames(VaR_bd) <- c("Total","Fondeo","Guber","Desc")
rownames(VaR_bd) <- c("VaR","CVaR")

print("VaR del BONDE")
print(VaR_bd)

[1] "VaR del BONDE"
         Total    Fondeo     Guber       Desc
VaR  -5.816561 -5.162887 -2.014550 -0.2722541
CVaR -8.652208 -7.998269 -2.697213 -0.5174403


## Futuros

### Tipo de Cambio

In [261]:
%%R
m=ncol(plazos_fowd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[5]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)

PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)

DeltaX_s_fwtdc=DeltaX_s[,sum(n_if[1:4],1):sum(n_if[1:5])]  #PASO CLAVE
x0_fwtdc=X_futtdc[1,] #PASO CLAVE

for (i in 1:Ns){
  X_s_fwtdc[i,]=x0_fwtdc*exp(DeltaX_s_fwtdc[i,])
  Vfr1_fwtdc[i,]=futuroTC(plazos_fowd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fowd*nominal_fwd
  Vfr2_fwtdc[i,]=futuroTC(plazos_fowd,x0_fwtdc[1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fowd*nominal_fwd
  Vfr3_fwtdc[i,]=futuroTC(plazos_fowd,x0_fwtdc[1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],X_s_fwtdc[i,(n_if[5])],kst_fwd)*contratos_fowd*nominal_fwd
 
  PGfr1_fwtdc[i,]=Vfr1_fwtdc[i,]-V0_fwtdc
  PGfr2_fwtdc[i,]=Vfr2_fwtdc[i,]-V0_fwtdc
  PGfr3_fwtdc[i,]=Vfr3_fwtdc[i,]-V0_fwtdc
  PG_fwtdc[i,]=PGfr1_fwtdc[i,]+PGfr2_fwtdc[i,]+PGfr3_fwtdc[i,]
}

VaRTotal_ftdc=quantile(PG_fwtdc,1-alpha,Ns)
VaRTotalfr1_ftdc=quantile(PGfr1_fwtdc,1-alpha,Ns)
VaRTotalfr2_ftdc=quantile(PGfr2_fwtdc,1-alpha,Ns)
VaRTotalfr3_ftdc=quantile(PGfr3_fwtdc,1-alpha,Ns)

CVaRTotal_ftdc= mean(merge(which(PG_fwtdc<VaRTotal_ftdc),cbind(seq(1,Ns),PG_fwtdc), by.x=1,by.y=1)[,2])
CVaRTotalfr1_ftdc= mean(PGfr1_fwtdc[which(PGfr1_fwtdc<VaRTotalfr1_ftdc),])
CVaRTotalfr2_ftdc= mean(PGfr2_fwtdc[which(PGfr2_fwtdc<VaRTotalfr2_ftdc),])
CVaRTotalfr3_ftdc= mean(PGfr3_fwtdc[which(PGfr3_fwtdc<VaRTotalfr3_ftdc),])

In [262]:
%%R
VaR_ftdc <- rbind(cbind(VaRTotal_ftdc,VaRTotalfr1_ftdc,VaRTotalfr2_ftdc,VaRTotalfr3_ftdc),cbind(CVaRTotal_ftdc,CVaRTotalfr1_ftdc,CVaRTotalfr2_ftdc,CVaRTotalfr3_ftdc))
colnames(VaR_ftdc) <- c("Total","Tasa Ext","Tasa Dom","TDC")
rownames(VaR_ftdc) <- c("VaR","CVaR")

print("VaR total y por factor de riesgos")
print(VaR_ftdc)

[1] "VaR total y por factor de riesgos"
         Total     Tasa Ext    Tasa Dom       TDC
VaR  -4.365900 -0.002087720 -0.02178657 -4.365715
CVaR -5.256553 -0.002638632 -0.02776965 -5.254852


### Indices

In [263]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fwd_ind)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[6]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
Vfr1_fwind=matrix(0,Ns,m)#Dividendos
Vfr2_fwind=matrix(0,Ns,m)#Tasa guver
Vfr3_fwind=matrix(0,Ns,m)#IPC

PGT_fwind=matrix(0,Ns,1)
PGfr1T_fwind=matrix(0,Ns,1)
PGfr2T_fwind=matrix(0,Ns,1)
PGfr3T_fwind=matrix(0,Ns,1)

DeltaX_s_fwind=DeltaX_s[,sum(n_if[1:5],1):sum(n_if[1:6])]  #PASO CLAVE
x0_fwind=X_futind[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_fwind[i,]=x0_fwind* exp(DeltaX_s_fwind[i,])
  Vfr1_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],x0_fwind[(n_if[6]/3+1):(n_if[6]*2/3)],x0_fwind[(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  Vfr2_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],x0_fwind[(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  Vfr3_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[6]/3)],x0_fwind[(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
 
  PGfr1T_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2T_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3T_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
  PGT_fwind[i,] = PGfr1T_fwind[i,]+PGfr2T_fwind[i,]+PGfr3T_fwind[i,]
}

VaRTotal_find=quantile(PGT_fwind,1-alpha,Ns)
VaRTotalfr1_find=quantile(PGfr1T_fwind,1-alpha,Ns)
VaRTotalfr2_find=quantile(PGfr2T_fwind,1-alpha,Ns)
VaRTotalfr3_find=quantile(PGfr3T_fwind,1-alpha,Ns)

CVaRTotal_find= mean(merge(which(PGT_fwind<VaRTotal_find),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,2])
CVaRTotalfr1_find= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_find),])
CVaRTotalfr2_find= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_find),])
CVaRTotalfr3_find= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_find),])

In [264]:
%%R
VaR_find <- rbind(cbind(VaRTotal_find,VaRTotalfr1_find,VaRTotalfr2_find,VaRTotalfr3_find),cbind(CVaRTotal_find,CVaRTotalfr1_find,CVaRTotalfr2_find,CVaRTotalfr3_find))
colnames(VaR_find) <- c("Total","Dividendos","Guber","Ind")
rownames(VaR_find) <- c("VaR","CVaR")

print("VaR total y por factor de riesgos")
print(VaR_find)

[1] "VaR total y por factor de riesgos"
         Total Dividendos     Guber       Ind
VaR  -32396.09          0 -180.9219 -32354.20
CVaR -36707.17        NaN -221.1787 -36633.41


## Swap's

In [265]:
%%R
#Cálculo de matriz de pérdidas y ganancias SWAP

#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[7]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)#Descuento
Vfr2_sw=matrix(0,Ns,m)#Cupon
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:6],1):sum(n_if[1:7])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*(1+DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[7]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[7]/2+1):(n_if[7])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[7]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[7]/2+1):(n_if[7])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}

#VaR por posición
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)
CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_sw[i]=quantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=quantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=quantile(PGfr2_sw[,i],1-alpha,Ns)
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total
VaRTotal_sw=quantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=quantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=quantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])

In [266]:
%%R
VaR_sw <- rbind(cbind(VaRCont_sw[,1],VaRfr1_sw[,1],VaRfr2_sw[,1],VaRCont_sw[,2],VaRfr1_sw[,2],VaRfr2_sw[,2]),
                cbind(CVaRCont_sw[,1],CVaRfr1_sw[,1],CVaRfr2_sw[,1],CVaRCont_sw[,2],CVaRfr1_sw[,2],CVaRfr2_sw[,2]))
colnames(VaR_sw) <- c("Total","Descuento","Cupon","Total","Descuento","Cupon")
rownames(VaR_sw) <- c("VaR","CVaR")

print("VaR Total y por factor de riesgo del Swap Largo")
print(VaR_sw[,1:3])

print("VaR Total y por factor de riesgo del Swap Corot")
print(VaR_sw[,4:6])

VaRT_sw <- rbind(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw),cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))
colnames(VaRT_sw) <- c("Total","Descuento","Cupon")
rownames(VaRT_sw) <- c("VaR","CVaR")

print("VaR Total y por factor de riesgo")
print(VaRT_sw)

[1] "VaR Total y por factor de riesgo del Swap Largo"
         Total Descuento     Cupon
VaR  -27861.93 -27865.91 -74.59878
CVaR -34080.83 -34001.02 -91.13510
[1] "VaR Total y por factor de riesgo del Swap Corot"
         Total Descuento     Cupon
VaR  -25438.41 -25552.58 -31.92441
CVaR -37108.18 -37298.96 -39.56063
[1] "VaR Total y por factor de riesgo"
         Total Descuento     Cupon
VaR  -19838.10 -19815.26 -105.4019
CVaR -24209.23 -24137.70 -129.4100


## Opciones

In [267]:
%%R
#Cálculo de matriz de pérdidas y ganancias Opciones Tasa de interés
#dimensión
m=ncol(plazos_opc)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[8]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s[,sum(n_if[1:7],1):sum(n_if[1:8])]  #PASO CLAVE
x0_oir=X_opc[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*(1+DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[8]/3))],X_s_oir[i,((n_if[8]/3+1):(n_if[8]/3*2))],K_opc,X_s_oir[i,((n_if[8]*2/3+1):(n_if[8]))],plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[8]/3))],x0_oir[((n_if[8]/3+1):(n_if[8]/3*2))],K_opc,x0_oir[((n_if[8]*2/3+1):(n_if[8]))],plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[8]/3))],X_s_oir[i,((n_if[8]/3+1):(n_if[8]/3*2))],K_opc,x0_oir[((n_if[8]*2/3+1):(n_if[8]))],plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[8]/3))],x0_oir[((n_if[8]/3+1):(n_if[8]/3*2))],K_opc,X_s_oir[i,((n_if[8]*2/3+1):(n_if[8]))],plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_opc
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_opc
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_opc
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_opc
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}

#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=quantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=quantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=quantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=quantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total
VaRTotal_oir=quantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=quantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=quantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=quantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])

In [268]:
%%R
VaR_oir <- rbind(cbind(VaRCont_oir[,1],VaRfr1_oir[,1],VaRfr2_oir[,1],VaRfr3_oir[,1],VaRCont_oir[,2],VaRfr1_oir[,2],VaRfr2_oir[,2],VaRfr3_oir[,1]),
                cbind(CVaRCont_oir[,1],CVaRfr1_oir[,1],CVaRfr2_oir[,1],CVaRfr3_oir[,1],CVaRCont_oir[,2],CVaRfr1_oir[,2],CVaRfr2_oir[,2],VaRfr3_oir[,1]))
colnames(VaR_oir) <- c("Total","Descuento","Spot","Volatildiad","Total","Descuento","Spot","Volatildiad")
rownames(VaR_oir) <- c("VaR","CVaR")

print("VaR Total y por factor de riesgo de la Opcion Call Larga")
print(VaR_oir[,1:4])

print("VaR Total y por factor de riesgo de la Opcion Put Larga")
print(VaR_oir[,5:8])

VaRT_oir <- rbind(cbind(VaRTotal_oir,VaRTotalfr1_oir,VaRTotalfr2_oir,VaRTotalfr3_oir),
                 cbind(CVaRTotal_oir,CVaRTotalfr1_oir,CVaRTotalfr2_oir,CVaRTotalfr3_oir))
colnames(VaRT_oir) <- c("Total","Descuento","Spot","Volatilidad")
rownames(VaRT_oir) <- c("VaR","CVaR")

print("VaR Total y por factor de riesgo")
print(VaRT_oir)

[1] "VaR Total y por factor de riesgo de la Opcion Call Larga"
           Total    Descuento        Spot Volatildiad
VaR  -0.05957620 -0.006536315 -0.03845898 -0.03755833
CVaR -0.08474503 -0.009587446 -0.04566180 -0.06294256
[1] "VaR Total y por factor de riesgo de la Opcion Put Larga"
           Total    Descuento        Spot Volatildiad
VaR  -0.04911466 -0.001624584 -0.03627153 -0.03755833
CVaR -0.06947778 -0.002375747 -0.05190024 -0.03755833
[1] "VaR Total y por factor de riesgo"
           Total    Descuento        Spot Volatilidad
VaR  -0.07335407 -0.008159581 -0.02765493 -0.06277878
CVaR -0.12356227 -0.011962961 -0.03386247 -0.01934634


## Riesgo Integral

In [269]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
#2. Forward de IPC
PGPort_ACC=PGfr2T_acc_div+ PGfr3T_fwind #Pérdidas y ganancias
VaRPort_ACC=quantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR

#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGT_bd+PGT_sw+PGfr1_fwtdc+PGfr2_fwtdc +PGfr2T_fwind+PGfr1T_fwind +PGfr1T_oir +PGfr2T_oir #Pérdidas y ganancias
VaRPort_TI=quantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr1T_acc_div+PGfr3_fwtdc  #Pérdidas y ganancias
VaRPort_TDC=quantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés
PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias
VaRPort_VOL=quantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR

#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL
VaRTotal_Port=quantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR

In [270]:
%%R
VaRT_port <- rbind(cbind(VaRTotal_Port,VaRPort_ACC,VaRPort_TI,VaRPort_TDC,VaRPort_VOL),cbind(CVaRTotal_Port,CVaRPort_ACC,CVaRPort_TI,CVaRPort_TDC,CVaRPort_VOL))
colnames(VaRT_port) <- c("Total","Acciones","Tasas de Interes","Tipo de Cambio","Volatilidad")
rownames(VaRT_port) <- c("VaR","CVaR")

print("VaR del portafolio")
print(VaRT_port)

[1] "VaR del portafolio"
         Total  Acciones Tasas de Interes Tipo de Cambio Volatilidad
VaR  -34931.24 -29258.03        -19704.94      -72.93165 -0.06277878
CVaR -40953.01 -33205.45        -24074.86      -87.58130 -0.10962148
